# **Lab 1: APIs, Google Cloud Functions, Docker and cloud Run:**



---



# **Exercise 2: Google Cloud Functions:**

- **0.** Check to see that the API key works correctly
- **1.** Write a function on GCP in Python
- **2.** Call the function from a Notebook (either in Colab or in Jupyter).


## **0. Check to see that the API key works correctly**

**Goal**: The goal of these two functions are to:
- **Function 1:** Fetch a list of movies
- **Function 2:** Fetch the poster of a specific movie ID

In [96]:
api_key = 'YOUR_API_KEY'
r = requests.get('https://api.themoviedb.org/3/movie/464052?api_key=' + api_key)
obj = json.loads(r.text)
movie_name = obj['title']
print(movie_name)

Wonder Woman 1984


## **1. Write two functions on GCP in Python**

### **Function 1:** Fetch a list of movies

**Note:** The same code can be used on Google Cloud, you just need to modify the three following lines:
- Add: import functions_framework
- Add: @functions_framework.http
- Add: request to the function: def get_movie_titles(request):

You should also add in the requirements.txt the two following libraries:
- functions-framework==3.*
- requests==2.31.0


In [113]:
#import functions_framework
import requests
from flask import jsonify

# @functions_framework.http
def get_movie_titles(): # request
    try:
        movie_database_url = 'https://api.themoviedb.org/3/discover/movie'
        api_key = 'YOUR_API_KEY'

        response = requests.get(movie_database_url, params={'api_key': api_key, 'page': 1})

        # Check if the request was successful (status code 200)
        response.raise_for_status()

        # Parse the JSON response
        data = response.json()

        # Check if there are results
        if 'results' in data and data['results']:
            # Filter movies that have a poster path
            movies_with_poster = [movie for movie in data['results'] if 'poster_path' in movie]

            movie_details = [{'title': movie['title'], 'id': movie['id']} for movie in movies_with_poster]

            # Return a JSON response with the list of movie details
            return {'movie_details': movie_details}
        else:
            return {'error': 'No results found.'}, 404
    except requests.RequestException as e:
        # Log the error details
        print(f"Error making API request: {e}")
        return 'Internal Server Error', 500
    except requests.exceptions.HTTPError as e:
        # Log the HTTP error details
        print(f"HTTP Error: {e}")
        return 'Internal Server Error', 500
    except json.JSONDecodeError as e:
        # Log the error details
        print(f"Error decoding JSON response: {e}")
        return 'Internal Server Error', 500


In [114]:
# Call the function and print the result
result = get_movie_titles()
result

{'movie_details': [{'title': 'Badland Hunters', 'id': 933131},
  {'title': 'Sixty Minutes', 'id': 1212073},
  {'title': 'Wonka', 'id': 787699},
  {'title': 'The Marvels', 'id': 609681},
  {'title': 'The Beekeeper', 'id': 866398},
  {'title': 'Lift', 'id': 955916},
  {'title': 'Aquaman and the Lost Kingdom', 'id': 572802},
  {'title': 'Migration', 'id': 940551},
  {'title': 'Wish', 'id': 1022796},
  {'title': 'The Hunger Games: The Ballad of Songbirds & Snakes',
   'id': 695721},
  {'title': 'Orion and the Dark', 'id': 1139829},
  {'title': 'The Family Plan', 'id': 1029575},
  {'title': 'Rebel Moon - Part One: A Child of Fire', 'id': 848326},
  {'title': "The Tiger's Apprentice", 'id': 598387},
  {'title': 'Role Play', 'id': 848187},
  {'title': 'The Bricklayer', 'id': 927107},
  {'title': 'Silent Night', 'id': 891699},
  {'title': 'Freelance', 'id': 897087},
  {'title': 'The Painter', 'id': 1211957},
  {'title': 'Anyone But You', 'id': 1072790}]}

### **Function 2:** Fetch the poster of a specific Movie


#### **Main.py**

In [111]:
import functions_framework
from flask import jsonify
import requests

api_key = 'YOUR_API_KEY'


@functions_framework.http
def get_movie_details(request):
    try:
        # Extract the movie ID from the request parameters
        movie_id = request.args.get('movie_id')

        # Make the API request using the provided movie ID
        url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}'
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        response.raise_for_status()

        # Parse the JSON response
        obj = response.json()

        poster_path_prefix = 'https://image.tmdb.org/t/p/w1280/'
        poster_path = f"{poster_path_prefix}{obj.get('poster_path')}"


        # Return a JSON response with the movie details
        response_data = {'movie_name': obj["title"], 'movie_poster': poster_path}
        return jsonify(response_data)
    except requests.RequestException as e:
        # Log the error details
        print(f"Error making API request: {e}")
        return 'Internal Server Error', 500
    except requests.exceptions.HTTPError as e:
        # Log the HTTP error details
        print(f"HTTP Error: {e}")
        return 'Internal Server Error', 500
    except json.JSONDecodeError as e:
        # Log the error details
        print(f"Error decoding JSON response: {e}")
        return 'Internal Server Error', 500


#### Requirements.txt

- functions-framework==3.*
- requests==2.31.0

## **2. Deploy the function on Google Cloud Platform**

## **3. Call the function from a Notebook (either in Colab or in Jupyter)**

In [102]:
# Testing function 1:
r = requests.get('YOUR_URL')
obj = json.loads(r.text)
print(obj)


# Testing function 2:
movie_id = '933131'
r = requests.get(f'YOUR_URL?movie_id={movie_id}')
obj = json.loads(r.text)
print(obj)

{'movie_details': [{'id': 933131, 'title': 'Badland Hunters'}, {'id': 1212073, 'title': 'Sixty Minutes'}, {'id': 787699, 'title': 'Wonka'}, {'id': 609681, 'title': 'The Marvels'}, {'id': 866398, 'title': 'The Beekeeper'}, {'id': 955916, 'title': 'Lift'}, {'id': 572802, 'title': 'Aquaman and the Lost Kingdom'}, {'id': 940551, 'title': 'Migration'}, {'id': 1022796, 'title': 'Wish'}, {'id': 695721, 'title': 'The Hunger Games: The Ballad of Songbirds & Snakes'}, {'id': 1139829, 'title': 'Orion and the Dark'}, {'id': 1029575, 'title': 'The Family Plan'}, {'id': 848326, 'title': 'Rebel Moon - Part One: A Child of Fire'}, {'id': 598387, 'title': "The Tiger's Apprentice"}, {'id': 848187, 'title': 'Role Play'}, {'id': 927107, 'title': 'The Bricklayer'}, {'id': 891699, 'title': 'Silent Night'}, {'id': 897087, 'title': 'Freelance'}, {'id': 1211957, 'title': 'The Painter'}, {'id': 1072790, 'title': 'Anyone But You'}]}
{'movie_name': 'Badland Hunters', 'movie_poster': 'https://image.tmdb.org/t/p/w1